In [18]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from google.colab import drive

drive.flush_and_unmount()

drive.mount('/content/drive', force_remount=True)

directory_path = '/content/drive/MyDrive/CS7643FinalProject'
if os.path.exists(directory_path):
    os.chdir(directory_path)
    print(f"Current working directory: {os.getcwd()}")
else:
    print(f"Error: Directory {directory_path} does not exist.")

Mounted at /content/drive
Current working directory: /content/drive/MyDrive/CS7643FinalProject


In [19]:
!pip install trimesh

In [20]:
import torch
from point_pillar.pillar_model import PointPillarsClassifier
from point_pillar.pillar_trainer import Trainer
from torch.utils.data import DataLoader
from point_pillar.modelnet_dataset import ModelNetDataset
from point_pillar.modelnet_dataset import augment_pointcloud
from point_pillar.config import config

In [21]:
import random
import numpy as np

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Enforce deterministic behaviour
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Extra safety for dataloader workers
    os.environ["PYTHONHASHSEED"] = str(seed)

def seed_worker(worker_id):
    import numpy as np
    import random
    seed = 42 + worker_id
    np.random.seed(seed)
    random.seed(seed)

set_seed(42)


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [23]:
config["train"]["num_epochs"] = 80
config["backbone"]["dropout_p"] = 0.15
config["dataset"]["num_points"] = 1024
config["dataset"]["name"] = 'ModelNet40'
config["dataset"]["num_classes"] = 40
print(config)

{'dataset': {'name': 'ModelNet40', 'num_classes': 40, 'num_points': 1024}, 'voxelizer': {'x_range': (-1.0, 1.0), 'y_range': (-1.0, 1.0), 'z_range': (-1.0, 1.0), 'pillar_size': (0.1, 0.1), 'max_pillars': 1024, 'max_points_per_pillar': 32}, 'pfn': {'in_dim': 8, 'out_dim': 64, 'hidden_dim': [64, 64], 'use_residual': True}, 'backbone': {'base_channels': 32, 'fc1_dim': 256, 'dropout_p': 0.15}, 'train': {'batch_size': 32, 'lr': 0.001, 'weight_decay': 0.0001, 'num_epochs': 80}}


In [24]:
DATA_DIR = "./data" + "/" + config["dataset"]["name"]
PREDATA_DIR = "./data" + "/" + config["dataset"]["name"] + "_precomputed_" + str(config["dataset"]["num_points"])

# import os
# import urllib.request
# import tarfile

os.makedirs(DATA_DIR, exist_ok=True)

# MODELNET_URL = "http://modelnet.cs.princeton.edu/ModelNet40.zip"  # or 40

# zip_path = f"{DATA_DIR}/ModelNet40.zip"
# if not os.path.exists(zip_path):
#     urllib.request.urlretrieve(MODELNET_URL, zip_path)

# # Unzip
# !unzip -q "{DATA_DIR}/ModelNet40.zip" -d "./data"

train_dataset = ModelNetDataset(
    root=DATA_DIR,
    split="train",
    num_points=config["dataset"]["num_points"],
    normalize=True,
    precomputed_root=PREDATA_DIR,
    cache_mode="write",
    transform=augment_pointcloud
)
val_dataset = ModelNetDataset(
    root=DATA_DIR,
    split="test",
    num_points=config["dataset"]["num_points"],
    normalize=True,
    precomputed_root=PREDATA_DIR,
    cache_mode="write",
    transform=augment_pointcloud
)

Found 40 classes, 9843 samples
Found 40 classes, 2468 samples


In [25]:
# for i in range(len(train_dataset)):
#     _ = train_dataset[i]
# for i in range(len(val_dataset)):
#     _ = val_dataset[i]

In [26]:
# 1. build datasets / loaders using config
train_dataset = ModelNetDataset(
    root=DATA_DIR,
    split="train",
    num_points=config["dataset"]["num_points"],
    normalize=True,
    precomputed_root=PREDATA_DIR,
    cache_mode="load",
    transform=augment_pointcloud
)
val_dataset = ModelNetDataset(
    root=DATA_DIR,
    split="test",
    num_points=config["dataset"]["num_points"],
    normalize=True,
    precomputed_root=PREDATA_DIR,
    cache_mode="load",
    transform=augment_pointcloud
)

Found 40 classes, 9843 samples
Found 40 classes, 2468 samples


In [27]:
# # 1. build datasets / loaders using config
# train_dataset = ModelNetDataset(
#     root=DATA_DIR,
#     split="train",
#     num_points=config["dataset"]["num_points"],
#     normalize=True,
#     precomputed_root=None,
#     cache_mode="load",
# )
# val_dataset = ModelNetDataset(
#     root=DATA_DIR,
#     split="test",
#     num_points=config["dataset"]["num_points"],
#     normalize=True,
#     precomputed_root=None,
#     cache_mode="load",
# )

In [28]:
g = torch.Generator()
g.manual_seed(42)

train_loader = DataLoader(
    train_dataset,
    batch_size=config["train"]["batch_size"],
    shuffle=True,
    num_workers=0,
    worker_init_fn=seed_worker,
    generator=g,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=config["train"]["batch_size"],
    shuffle=False,
    num_workers=0,
    worker_init_fn=seed_worker,
    generator=g,
)

In [29]:
# 2. build model from config
model = PointPillarsClassifier(config=config, device=device)

In [30]:

# 3. build trainer from config
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    config=config,
)

In [ ]:
# 4. train
# 9m 53s without cache 1 epoch
# 9m 13s with cache 1 epoch
# 5m 18s with voxelizer vectorize level 1
dataset_name = config["dataset"]["name"]
# add timestamp to the name of the output directory
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"output_{timestamp}"
os.makedirs(f"{output_dir}", exist_ok=True)
trainer.fit(save_path=f"{output_dir}/pointpillars_{dataset_name}.pth")

[Epoch 1 Batch 0] Loss=3.7298 Acc=0.031
[Epoch 1 Batch 20] Loss=2.9335 Acc=0.250
[Epoch 1 Batch 40] Loss=2.4507 Acc=0.469
[Epoch 1 Batch 60] Loss=1.6637 Acc=0.594
[Epoch 1 Batch 80] Loss=2.0354 Acc=0.469
[Epoch 1 Batch 100] Loss=1.4405 Acc=0.500
[Epoch 1 Batch 120] Loss=1.3626 Acc=0.625
[Epoch 1 Batch 140] Loss=1.3031 Acc=0.594
[Epoch 1 Batch 160] Loss=1.0181 Acc=0.719
[Epoch 1 Batch 180] Loss=1.2163 Acc=0.625
[Epoch 1 Batch 200] Loss=1.3490 Acc=0.594
[Epoch 1 Batch 220] Loss=1.0817 Acc=0.656
[Epoch 1 Batch 240] Loss=0.8569 Acc=0.750
[Epoch 1 Batch 260] Loss=1.0465 Acc=0.688
[Epoch 1 Batch 280] Loss=1.0771 Acc=0.625
[Epoch 1 Batch 300] Loss=0.6365 Acc=0.719
--> Train Epoch 1: Loss=1.6216, Acc=0.5536
--> Val   Epoch 1: Loss=1.1132, Acc=0.6779
  [*] Saved new best model (val_acc=0.6779)
[Epoch 2 Batch 0] Loss=1.0467 Acc=0.656
[Epoch 2 Batch 20] Loss=1.0629 Acc=0.719
[Epoch 2 Batch 40] Loss=0.5344 Acc=0.875
[Epoch 2 Batch 60] Loss=1.1168 Acc=0.594
[Epoch 2 Batch 80] Loss=0.7834 Acc=0.781


In [ ]:
# 5. plots for report
trainer.plot_history()

In [ ]:
# 6. save artifacts

trainer.save_curves_and_config(output_dir=output_dir)

In [ ]:
import csv

new_row = {

    "timestamp": timestamp,
    "name": config["dataset"]["name"],
    "num_classes": config["dataset"]["num_classes"],
    "num_points": config["dataset"]["num_points"],
    "pillar_size": config["voxelizer"]["pillar_size"],
    "max_pillars": config["voxelizer"]["max_pillars"],
    "max_points_per_pillar": config["voxelizer"]["max_points_per_pillar"],
    "pfn_out_dim": config["pfn"]["out_dim"],
    "bb_base_channels": config["backbone"]["base_channels"],
    "bb_fc1_dim": config["backbone"]["fc1_dim"],
    "bb_dropout_p": config["backbone"]["dropout_p"],
    "batch_size": config["train"]["batch_size"],
    "lr": config["train"]["lr"],
    "weight_decay": config["train"]["weight_decay"],
    "num_epochs": config["train"]["num_epochs"],
    "min_train_loss": min(trainer.history["train_loss"]),
    "min_val_loss": min(trainer.history["val_loss"]),
    "max_train_acc": max(trainer.history["train_acc"]),
    "max_val_acc": max(trainer.history["val_acc"])
}

csv_path = "config_log.csv"

# Append mode
with open(csv_path, "a", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=new_row.keys())

    # Write header only if file is empty
    if f.tell() == 0:
        writer.writeheader()

    writer.writerow(new_row)
